In [2]:
import numpy as np
import gdal
import glob
import os
import datetime
from scipy.interpolate import griddata
import re
import xarray as xr

In [23]:
# Collect Files
file_dict = {
    'EVI' : glob.glob('MODIS_NDVI/MOD13*EVI*.tif'),
    'EVI_QC' : glob.glob('MODIS_NDVI/*pixel_reliability*.tif'),
    'TA' : glob.glob('MODIS_TA/MOD14*FireMask*.tif'),
    'TA_QC' : glob.glob('MODIS_TA/*QA*.tif'),
    'LST_Day' : glob.glob('MODIS_LST_Day/*LST_Day*.tif'),
    'LST_Day_QC' : glob.glob('MODIS_LST_Day/*QC_Day*.tif'),
    'LST_Night' : glob.glob('MODIS_LST_Night/*LST_Night*.tif'),
    'LST_Night_QC' : glob.glob('MODIS_LST_Night/*QC_Night*.tif')
}

In [24]:
# Open files and store as netcdfs
pat = r'doy(\d{7})'
for k, v in file_dict.items():
    da_list = []
    print(k)
    for fp in v:
        time = re.findall(pat, fp)[0]
        year = int(time[:4])
        day = int(time[-3:])
        da = xr.open_rasterio(fp)
        dt = (datetime.datetime(year, 1, 1) + datetime.timedelta(day - 1))
        da = da.assign_coords(time=dt)
        da_list.append(da)
    da = xr.concat(da_list, dim='time').squeeze()
    da.name = k
    da.to_netcdf('netcdfs/%s.nc' % k)
    print(k, 'Done')


EVI
EVI Done
EVI_QC
EVI_QC Done
TA
TA Done
TA_QC
TA_QC Done
LST_Day
LST_Day Done
LST_Day_QC
LST_Day_QC Done
LST_Night
LST_Night Done
LST_Night_QC
LST_Night_QC Done
